
### The problem 

The location data is latitude and longitude. I want to cluster together certains locations together and set new categories based on those clusters.

Geohashing is one technique, but the underlying geometry is not perfectly euclidean (We are dealing with a globe after all), I have want to incorporate geodesic geometry. 

### The solution
To do this I will first geohash the data and one hot encode it and run ols. Then, I will do the same thing using hdbscan


#### I need to test the geohashing method ASAP! 

There is a good chance that I won't be able to match up the geo tags for the predictions.. 
I wonder how well this will work.

https://en.wikipedia.org/wiki/Geohash#Design

In [3]:
from sklearn.feature_selection import RFE
from sklearn.neighbors import KNeighborsClassifier
import geohash2
import hdbscan
import pandas as pd
import numpy as np

In [3]:
# points = np.array([[lat1, lon1], [lat2, lon2], ...])
# rads = np.radians(points)
# clusterer = hdbscan.HDBSCAN(min_cluster_size=N, metric='haversine')
# cluster_labels = clusterer.fit_predict(points)

In [5]:
df  = pd.read_csv("../raw/kc_house_data_train.csv", index_col=0)

In [6]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2591820310,20141006T000000,365000.0,4,2.25,2070,8893,2.0,0,0,...,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700
1,7974200820,20140821T000000,865000.0,5,3.00,2900,6730,1.0,0,0,...,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283
2,7701450110,20140815T000000,1038000.0,4,2.50,3770,10893,2.0,0,2,...,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685
3,9522300010,20150331T000000,1490000.0,3,3.50,4560,14608,2.0,0,2,...,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226
4,9510861140,20140714T000000,711000.0,3,2.50,2550,5376,2.0,0,0,...,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050


## Create a pd dataframe using haversine density scanning 

In [7]:
# create a np array called pointer containg the latitude and longitude of each obs
points = np.array(list(zip(df.lat, df.long)))

In [9]:
len(points) == len(df)

True

In [11]:
# Turn the points from degrees to radians
points = np.radians(points)
points[0]

array([ 0.01445068, -0.03721267])

In [107]:
# instantiate our haversine clustering object 
N = 15
min_size = 5
clusterer = hdbscan.HDBSCAN(min_cluster_size=N, metric='haversine')

In [108]:
# fit the clusterer to our data
cluster_labels = clusterer.fit_predict(points)

In [109]:
clusters = cluster_labels.tolist()
clusters = pd.DataFrame(pd.Series(clusters))
clusters.columns = ['hdb_clusters']

## Make dummy columns

In [116]:
clusters = pd.get_dummies(clusters.hdb_clusters)

In [117]:
clusters.to_csv("../cleanedData/hdb_clusters.csv")

In [102]:
neigh = KNeighborsClassifier(n_neighbors=5)

In [69]:
neigh.fit(points)

TypeError: fit() missing 1 required positional argument: 'y'

## Get holdout geohashing data

In [5]:
df = pd.read_csv('../raw/kc_holdout_data.csv')

In [8]:
points = np.array(list(zip(df.lat, df.long)))
points = np.radians(points)

In [9]:
# Create a function to apply the geohashing function to the lat and long columns.

def geo_hash(lat, long):
    return geohash2.encode(lat, long, 5)

# pull out the latitude and longitude data
location = df[['lat', 'long']]

location['hashed_location'] = location.apply(lambda x: geo_hash(x['lat'], x['long']), axis=1)
# apply the geo_has function to the latitude and longitude columns of the dataframe
# location.apply(geo_hash(location), axis=1)

/Users/TjH/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [10]:
location

,lat,long,hashed_location
0,47.7089,-122.241,c23p6
1,47.7089,-122.241,c23p6
2,47.5472,-121.998,c23nr
3,47.7427,-122.071,c23pw
4,47.4863,-122.140,c23nh
...,...,...,...
4318,47.6993,-122.346,c22zr
4319,47.5107,-122.362,c22yr
4320,47.5944,-122.299,c23nc
4321,47.5345,-122.069,c23nq


# Insure holdout and regular have the same number of dummies

In [11]:

dummy_geo = pd.get_dummies(location.hashed_location)

In [21]:
set_holdout_geo = set(dummy_geo.columns)

In [22]:
geo = pd.read_csv('../cleanedData/geohashed_full.csv', index_col=0)

In [23]:
set_geo = set(geo.columns)

In [31]:
(set_holdout_geo.difference(set_geo))

{'c23hq'}

In [26]:
dummy_geo.shape, geo.shape

((4323, 178), (17290, 196))

In [27]:
196-178

18

In [34]:
overlap_holdout_geo = geo.drop(columns= list(set_geo.difference(set_holdout_geo)))

# Out by 19

In [35]:
overlap_holdout_geo.columns

Index(['c22uz', 'c22vm', 'c22vn', 'c22vp', 'c22vr', 'c22vs', 'c22vt', 'c22vu',
       'c22vv', 'c22vw',
       ...
       'c23r2', 'c23r3', 'c23r4', 'c23r6', 'c23r8', 'c23r9', 'c23rd', 'c23rf',
       'c23xm', 'c23xq'],
      dtype='object', length=177)

In [36]:
set(overlap_holdout_geo).difference(set_holdout_geo)

set()

In [57]:
overlap_holdout_geo.shape

(17290, 177)

In [38]:
overlap_holdout_geo.columns

Index(['c22uz', 'c22vm', 'c22vn', 'c22vp', 'c22vr', 'c22vs', 'c22vt', 'c22vu',
       'c22vv', 'c22vw',
       ...
       'c23r2', 'c23r3', 'c23r4', 'c23r6', 'c23r8', 'c23r9', 'c23rd', 'c23rf',
       'c23xm', 'c23xq'],
      dtype='object', length=177)

In [44]:
len(set(list(dummy_geo.columns) + list(geo.columns)))

197

In [48]:
holdout_geo = dummy_geo.drop(columns=['c23hq'])

In [51]:
len(holdout_geo.columns)

177

In [56]:
# We have 177 overlapping hashed regions! Save to csv and let's gooooo

holdout_geo.to_csv('../cleanedData/holdoutGeo.csv')

In [54]:
df.shape

(4323, 21)

In [58]:
overlap_holdout_geo.to_csv('../cleanedData/geoOverlap.csv')